In [1]:
import cv2
import numpy as np

In [2]:
"""capture = cv2.VideoCapture(r"D:\User Profile Data\Desktop\prog\python\projects\Lane_Detection\test_video.mp4")
ret, frame = capture.read()

# Checking if the frame was read successfully
if ret:
    # Display the frame
    cv2.imshow('Frame', frame)
    cv2.waitKey(0)

# Release the VideoCapture object and close any open windows
capture.release()
cv2.destroyAllWindows()"""


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 31-32: truncated \UXXXXXXXX escape (799070177.py, line 12)

In [3]:
capture= cv2.VideoCapture(r"D:\User Profile Data\Desktop\prog\python\projects\Lane_Detection\test_video.mp4")

In [7]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges

def region_of_interest(image, vertices):
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, vertices, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def detect_lines(image, rho, theta, threshold, min_line_length, max_line_gap):
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    return lines

def draw_lines(image, lines, color=(0, 0, 255), thickness=2):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)

def lane_detection(image):
    height, width, _ = image.shape
    
    roi_vertices = np.array([[(0, height), (width // 2, height // 2), (width, height)]], dtype=np.int32)
    
    edges = preprocess_image(image)
    roi_edges = region_of_interest(edges, roi_vertices)
    
    lines = detect_lines(roi_edges, rho=1, theta=np.pi/180, threshold=40, min_line_length=100, max_line_gap=50)
    
    lane_image = np.zeros_like(image)
    draw_lines(lane_image, lines)
    
    result = cv2.addWeighted(image, 0.8, lane_image, 1, 0)
    return result

# Read the input video
cap = cv2.VideoCapture('test_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    result = lane_detection(frame)
    
    cv2.imshow('Lane Detection', result)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
